In [1]:
# -*- coding: utf-8 -*-
import os, sys
import json

import glob
sys.path.append("..")
sys.path.append("../..")
import dataset_utils.utils as utils
import dataset_utils.config as config
import cv2
import dlib
import openface

from utils.FaceAligner import FaceAligner

In [2]:
size = 96
indices = openface.AlignDlib.OUTER_EYES_AND_NOSE
src_dir = os.path.join(config.DS_DIALOG_PILOT, "transformed")
tgt_dir = os.path.join(config.DS_DIALOG_PILOT, "aligned_face")
# movie_files = ["omni_dialog_pilot_01.MP4"]
movie_files = ["omni_dialog_pilot_02_01.MP4", "omni_dialog_pilot_02_02.MP4"]
json_dir = os.path.join(config.DS_DIALOG_PILOT, "face_bb_and_landmarks")
aligner = FaceAligner()

In [ ]:
for movie_file in movie_files:
    movie_id = movie_file.split(".")[0]
    src_sub_dir = os.path.join(src_dir, movie_id)
    tgt_sub_dir = os.path.join(tgt_dir, movie_id)
    if not os.path.exists(tgt_sub_dir):
        os.makedirs(tgt_sub_dir)
    json_path = os.path.join(json_dir, "%s.json" % movie_id)
    with open(json_path, "r") as fr:
        bblms = json.load(fr)
        
    src_paths = sorted(glob.glob(os.path.join(src_sub_dir, "*.jpg")))
    for i, path in enumerate(src_paths):
        print("%d / %d" % (i+1, len(src_paths)))

        img_name = os.path.basename(path)
        img = utils.imread(path)
        print(img_name)
        print(bblms)
        if img_name in bblms:
            bb = bblms[img_name]['bb']
            bb_rec = dlib.rectangle(bb[0], bb[1], bb[2], bb[3])
            lms = bblms[img_name]['landmarks']
            aligned_img = aligner.align(img=img, bb=bb_rec, landmarks=lms, size=size, indices=indices)
            tgt_path = os.path.join(tgt_sub_dir, img_name)
            utils.imwrite(tgt_path, aligned_img)

utils.notify("@nogaken align_face done!!")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




2 / 5238
00000001_B.jpg


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1624 / 5238
00000812_B.jpg


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1642 / 5238
00000821_B.jpg


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3136 / 5238
00001568_B.jpg


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

